# Improvement Carvana_local_learning_prod_ready

## Подключение библиотек и загрузка датасета

In [ ]:
# Улучшения: 
# Убрал .cuda() из класса модели и добавил в даталоадеры перенос на gpu
# Добавил трэйсинг моделии и ее сохранение, загрузку модели
# Заменил лишний ресайз масок на передачу оригинолов масок
# Сделал предсказания с разным размером батча
# Нейтрализовал проявления хардкода
# Написал комментарии и пояснения
# Сделал submission при обучении на всем трерировочном датасете
# переписал pil на cv2 где это можно - скорость увеличилась
# Реализовал ресайз через albumintation - скорость увеличилась
# Сделал нормализацию через albumintation улучшений нет, только время увеличилось
# Проверил, что при нормализации маска не изменяется
# Изменил формат вывода при обучении и предсказании
# сделал predict на torchscript - скорость не выросла
# заменил при валидации и в predict интерполяцию масок с nearest на bilinear
# с onnx получилось медленне, видимо нужно отдельно устанавливать cuda и cudnn

In [ ]:
# Эксперименты с unet(mobilenet), unet(efficientnetb5), deeplabv3plus(mobilenet), deeplabv3plus(efficientnetb5)
# в transform попробовать другое кроме cv2.INTER_LINEAR и сравнить точность
# попробовать в predict не nearest для масок и сравнить точность (написать код и посмотреть как маска меняется
# при интерполяции nearest, linear, bilinear и т.д. и раcсчитать ошибки)
# Попробовать softdice loss + bce (как в dlcource.ai)
# разобраться, заморожены ли веса энкодера у smp моделей
# реализовать модель из https://github.com/lyakaap/Kaggle-Carvana-3rd-Place-Solution/blob/master/model_pytorch.py
# наиболее предпочтительным методом интерполяции является cv.INTER_AREA - попробовать при аугментации
# Взять deeplab обученный из torchvision.models.segmentation.segmentation и протестировать вместо предыдущих моделей
# с интерполяцией и на полном изображении - сравнить точность с 30 - эпоховой моделью. Быть может, уперлись в точность
# именно из-за интерполяции
# сделать predict 30 - эпоховой модели при linear и bilinear интерполяции, сравнить точность

In [ ]:
!pip install segmentation_models_pytorch

In [1]:
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np
import glob
import cv2
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import time
from torch.autograd import Variable
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.onnx
from torchinfo import summary

In [30]:
# Выполнять, если датасет не загружен
#!pip install -q kaggle
#!mkdir ~/.kaggle
#!cp ~/kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle competitions download -c carvana-image-masking-challenge
#!unzip ~/carvana-image-masking-challenge.zip ~/carvana_dataset/

#!unzip ~/carvana_dataset/train.zip -d ~/carvana_dataset/train
#!unzip ~/carvana_dataset/test.zip -d ~/carvana_dataset/test
#!unzip ~/carvana_dataset/train_masks.zip -d ~/carvana_dataset/train_masks

#!unzip ~/carvana_dataset/train_hq.zip -d ~/carvana_dataset/train_hq
#!unzip ~/carvana_dataset/test_hq.zip -d ~/carvana_dataset/test_hq

#!unzip ~/carvana_dataset/train_masks.csv.zip  ~/carvana_dataset/
#!unzip ~/carvana_dataset/sample_submission.csv.zip  ~/carvana_dataset/
#!unzip ~/carvana_dataset/metadata.csv.zip  ~/carvana_dataset/

#!rm ~/carvana-image-masking-challenge.zip
#!rm ~/carvana_dataset/test.zip
#!rm ~/carvana_dataset/train_masks.zip
#!rm ~/carvana_dataset/train.zip
#!rm ~/carvana_dataset/test_hq.zip
#!rm ~/carvana_dataset/train_hq.zip
#!rm ~/carvana_dataset/train_masks.csv.zip
#!rm ~/carvana_dataset/sample_submission.csv.zip
#!rm ~/carvana_dataset/metadata.csv.zip

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

In [6]:
!nvidia-smi

Thu Oct  7 15:03:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   51C    P8    14W / 170W |    363MiB / 12045MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Используемые функции

In [2]:
def get_data_csv(imgs_path: str = None, masks_path: str = None) -> pd.DataFrame:
    '''Функция получает на вход пути к директориям с изображениями и масками
    и генерирует датафрейм, содержащий имя изображений, их адреса и адреса
    соответствующих им масок
  
    Входные параметры:
    imgs_path: str - путь к директории с изображениями,
    masks_path: str - путь к директории с масками
    Возвращаемые значения:
    pd.DataFrame: data - dataframe, содержащий адреса изображений и соответствующих им масок'''

    assert (imgs_path != None) & (masks_path != None)
    # imgs_path or masks_path is equal None

    data_img = {}
    data_mask = {}
    data_img['imgs_path'] = []
    data_mask['masks_path'] = []
    data_img['imgs_path'] = list(glob.glob(imgs_path + "/*"))
    data_mask['masks_path'] = list(glob.glob(masks_path + "/*"))

    data_img = pd.DataFrame(data_img)
    data_mask = pd.DataFrame(data_mask)

    def file_name(x):
        return x.split("/")[-1].split(".")[0]

    data_img["file_name"] = data_img["imgs_path"].apply(lambda x: file_name(x))
    data_mask["file_name"] = data_mask["masks_path"].apply(lambda x: file_name(x)[:-5])

    data = pd.merge(data_img, data_mask, on = "file_name", how = "inner")

    return data

In [3]:
def get_train_test(source_df: pd.DataFrame, separate_feature: str = None, test_size: float = 0.25) -> pd.DataFrame:
    '''Функция разделяет source_df на две части с коэффициентом test_size
    по уникальным значениям separate_feature так, чтобы в новых датафреймах
    не было строк с одинаковыми значенияти из separate_feature

    Входные параметры:
    source_df: pd.DataFrame - датафрейм для разделения на train и test
    separate_feature: str - поле, по которому датафрейм будет разделен
    test_size: float - коэффициент разделения дтафрейма
    Возвращаемые значения:
    pd.DataFrame: data_train - датафрейм для тренировки
    pd.DataFrame: data_valid - датафрейм для валидации'''
  
    if (separate_feature != None) & (separate_feature in source_df.columns):
        train_cars, valid_cars = train_test_split(source_df[separate_feature].unique(), test_size=test_size, random_state=42)
        data_valid = source_df[np.isin(source_df[separate_feature].values, valid_cars)]
        data_train = source_df[np.isin(source_df[separate_feature].values, train_cars)]
        assert source_df.shape[0] == (data_valid.shape[0] + data_train.shape[0])
        assert np.isin(data_train[separate_feature].values, data_valid[separate_feature].values).sum() == 0
    else:
        data_train, data_valid = train_test_split(source_df, test_size=test_size)

    return data_train, data_valid


In [4]:
def DICE(logits: torch.Tensor, targets: torch.Tensor, treashold: float) -> float:
    '''Функция для вычисления DICE коэффициента для набора изображенй в формате torch.Tensor
    Входные параметры:
    logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
    targets: torch.Tensor - тензор из целевых целевых значений масок
    treashold: float - порог для определения класса точки в предсказанной точке
    Возвращаемые значения:
    score: float - значение DICE коэффициента для набора предсказанных масок'''
    
    smooth = 1
    num = targets.size(0)
    probs = torch.sigmoid(logits)
    outputs = torch.where(probs > treashold, 1, 0)
    m1 = outputs.view(num, -1)
    m2 = targets.view(num, -1)
    intersection = (m1 * m2)

    score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
    score = score.sum() / num
    return score

In [5]:
def tensor_to_rle(tensor: torch.Tensor) -> str:
    '''Функция принимает одну маску в тензорном формате, элементы которой
    имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
    Входные параметры:
    tensor: torch.Tensor - маска в тензорном формате
    Возвращаемые значения:
    rle_str: str - rle представление маски в строком виде'''
    
    # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
    # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
    # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
    # не сильно упадет)
    tensor = tensor.view(1, -1)
    tensor = tensor.squeeze(0)
    tensor[0] = 0
    tensor[-1] = 0
    rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
    rle[1::2] = rle[1::2] - rle[:-1:2]
    rle = rle.cpu().detach().numpy()
    rle_str = rle_to_string(rle)
    return rle_str

In [6]:
def numpy_to_rle(mask_image: np.ndarray) -> str:
    '''Функция принимает одну маску в формате массива numpy, элементы которой
    имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
    Входные параметры:
    mask_image: numpy.ndarray - маска в тензорном формате
    Возвращаемые значения:
    rle_str: str - rle представление маски в строковом виде'''
    
    # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
    # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
    # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
    # не сильно упадет)
    pixels = mask_image.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle_str = rle_to_string(runs)
    return rle_str

In [7]:
def rle_to_string(runs: torch.Tensor) -> str:
    '''Функция преобразует последовательноть чисел в тензоре runs
    в строковое представление этой последовательности
    Входные параметры:
    runs: torch.Tensor - последовательность чисел в тензорном формате
    Возвращаемые значения:
    rle_str: str - строковое представление последовательности чисел'''
    
    return ' '.join(str(x) for x in runs)

In [8]:
def mask_to_rle(mask_addr: str) -> str:
    '''Функция преобразует маску, имеющую адрес mask_addr и сохраненную в
    формате .gif, элементы которой имеют значения 0 и 1 в rle представление
    в строковом виде
    Входные параметры:
    mask_addr: str - адрес маски
    Возвращаемые значения:
    mask_rle: str - rle представление маски в строком виде
    '''
    
    mask = Image.open(mask_addr).convert('LA') # преобразование в серый
    mask = np.asarray(mask).astype('float')[:,:,0]
    mask = mask/255.0
    mask_rle = numpy_to_rle(mask)
    return mask_rle

## Используемые классы

In [9]:
class DiceMetric(nn.Module):
    '''Класс для вычисления DICE коэффициента для набора изображенй в формате torch.Tensor
    с заданным порогом для определния класса каждой точки изображения'''
    
    def __init__(self, treashold: float=0.5):
        '''treashold: float - порог для определения класса точки в предсказанной точке'''
        super(DiceMetric, self).__init__()
        self.treashold = treashold

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> float:
        '''Входные параметры:
        logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
        targets: torch.Tensor - тензор из целевых целевых значений масок
        Возвращаемые значения:
        score: float - значение DICE коэффициента для набора предсказанных масок'''
        with torch.no_grad():
            smooth = 1
            num = targets.size(0)
            probs = torch.sigmoid(logits)
            outputs = torch.where(probs > self.treashold, 1., 0.)
            m1 = outputs.view(num, -1)
            m2 = targets.view(num, -1)
            intersection = (m1 * m2)

            score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
            score = score.sum() / num
            return score

In [10]:
class SoftDiceLoss(nn.Module):
    '''Класс для вычисления DICE loss для набора изображенй в формате torch.Tensor'''
    def __init__(self):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> float:
        '''Входные параметры:
        logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
        targets: torch.Tensor - тензор из целевых целевых значений масок
        Возвращаемые значения:
        score: float - значение DICE loss для набора предсказанных масок'''
        smooth = 1
        num = targets.size(0)
        probs = torch.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score

In [11]:
class CustomDatasetForTrain(Dataset):
    '''Класс для создания тренировочных и валидационных датасетов'''
    def __init__(self, data_info: pd.DataFrame, device: str, transform: object, skip_mask: bool=False):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами изображений и масок
        device: str - имя устройства, на котором будут обрабатываться данные
        transform: object - список трансформации, которым будут подвергнуты изображения и маски
        skip_mask: bool - флаг, нужно ли генерировать исходную маску (без изменения размерности)
        Возвращаемые значения:
        объект класса CustomDatasetForTrain'''
        # Подаем подготовленный датафрейм
        self.data_info = data_info
        # Разделяем датафрейм на rgb картинки 
        self.image_arr = self.data_info.iloc[:,0]
        # и на сегментированные картинки
        self.mask_arr = self.data_info.iloc[:,2]
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        # Устройство, на котором будут находиться выходные тензоры
        self.device = device
        # Нужно ли пробрасывать маску изображения на выход без изменений
        self.skip_mask = skip_mask
        # Сохраняем преобразования данных
        self.transform = transform

    def __getitem__(self, index: int):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        img: torch.Tensor - тензорное представление изображения с размерностью out_shape
        mask_small: torch.Tensor - тензорное представление маски с исходной размерностью
        mask: torch.Tensor - тензорное представление изображения с размерностью out_shape 
        (возвращается если значение skip_mask равно True)'''
        image = cv2.imread(self.image_arr[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        
        # gif не открывается через open cv, поэтому используем для чтения PIL Image
        mask = Image.open(self.mask_arr[index])
        mask = np.asarray(mask)#.astype('float')
        
        transformed = self.transform(image=image, mask=mask)
        tr_image = transformed['image']
        tr_mask = transformed['mask']
        
        tr_image = tr_image.to(self.device).float()
        tr_mask = tr_mask.to(self.device).float().unsqueeze(0)

        
        # Если необходима исходная маска, то дополнительно возвращаем ее
        if self.skip_mask == True:
            mask = (torch.as_tensor(mask)).to(self.device).float().unsqueeze(0)
            return (tr_image, tr_mask, mask)
        else:
            return (tr_image, tr_mask)

    def __len__(self):
        return self.data_len

In [12]:
class CustomDatasetForTest(Dataset):
    '''Класс для создания тестовых датасетов'''
    def __init__(self, data_info, device: str, transform: object):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами и именами изображений
        device: str - имя устройства, на котором будут обрабатываться данные
        transform: object - список трансформации, которым будут подвергнуты изображения
        Возвращаемые значения:
        объект класса CustomDatasetForTest'''
        # Подаем наш подготовленный датафрейм
        self.data_info = data_info
        # Получаем адреса RGB изображений 
        self.image_addresses = self.data_info.iloc[:,0]
        # Получаем имена RGB изображений 
        self.image_names = self.data_info.iloc[:,1]
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        # Устройство, на котором будут находиться выходные тензоры
        self.device = device
        # Сохраняем преобразования данных
        self.transform = transform

    def __getitem__(self, index):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        img: torch.Tensor - тензорное представление изображения с размерностью out_shape
        mask_small: torch.Tensor - тензорное представление маски с исходной размерностью
        image_name: str - имя изображения'''
        image = cv2.imread(self.image_addresses[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        
        transformed = self.transform(image=image)
        tr_image = transformed['image']
        tr_image = tr_image.to(self.device).float()
        image_name = self.image_names[index]
    
        return (index, tr_image, image_name)

    def __len__(self):
        return self.data_len

In [13]:
class NeuralNetwork(nn.Module):
    '''Класс для создания работы с нейронной сетью для семантической сегментации Carvana'''
    def __init__(self, model: object):
        '''Конструктор класса
        Входные параметры:
        model: nn.Module - последовательность слоев или модель, через которую будут проходить данные
        Возвращаемые значения: 
        объект класса NeuralNetwork'''
        super(NeuralNetwork, self).__init__()
        self.model = model

    def forward(self, input_data):
        '''Функция прямого прохода через объкт класса
        Входные параметры:
        input_data: torch.Tensor - тензорное представление изображения
        Возвращаемые значения: 
        input_data: torch.Tensor - тензорное представление маски изображения'''
        output_data = self.model(input_data)
        return output_data
    
    @staticmethod
    def tensor_to_rle(tensor: torch.Tensor) -> str:
        '''Статический метод принимает одну маску в тензорном формате, элементы которой
        имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
        Входные параметры:
        tensor: torch.Tensor - маска в тензорном формате
        Возвращаемые значения:
        rle_str: str - rle представление маски в строковом виде'''
    
        # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
        # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
        # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
        # не сильно упадет)
        with torch.no_grad():
            tensor = tensor.view(1, -1)
            tensor = tensor.squeeze(0)
            tensor[0] = 0
            tensor[-1] = 0
            rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
            rle[1::2] = rle[1::2] - rle[:-1:2]
            rle = rle.cpu().detach().numpy()
            rle_str = NeuralNetwork.rle_to_string(rle)
            return rle_str
    
    @staticmethod
    def rle_to_string(runs: torch.Tensor) -> str:
        '''Функция преобразует последовательноть чисел в тензоре runs
        в строковое представление этой последовательности
        Входные параметры:
        runs: torch.Tensor - последовательность чисел в тензорном формате
        Возвращаемые значения:
        rle_str: str - строковое представление последовательности чисел'''
        return ' '.join(str(x) for x in runs)
    
    
    def fit(self, criterion: object, metric: object, optimizer: object, 
                  train_data_loader: DataLoader, valid_data_loader: DataLoader=None, epochs: int=1):
        '''Метод для обучения объекта класса
        Входные параметры:
        criterion: object - объект для вычисления loss
        metric: object - объект для вычисления метрики качества
        optimizer: object - оптимизатор
        train_data_loader: DataLoader - загрузчик данных для обучения
        valid_data_loader: DataLoader - загрузчик данных для валидации
        epochs: int - количество эпох обучения
        
        Возвращаемые значения:
        result: dict - словарь со значениями loss при тренировке, валидации и метрики при валидации 
        для каждой эпохи'''
        
        self.optimizer = optimizer
        epoch_train_losses = []
        epoch_valid_losses = []
        epoch_valid_metrics = []
        result = {}
        
        for epoch in range(epochs):
            self.model.train()
            time1 = time.time()
            running_loss =0.0
            train_losses = []
            for batch_idx, (data, labels) in enumerate(train_data_loader):
                data, labels = Variable(data), Variable(labels)        

                optimizer.zero_grad()
                outputs = self.model(data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                train_losses.append(loss.item())
                if (batch_idx+1) % 300 == 0:
                    print(f'Train Epoch: {epoch+1}, Loss: {(running_loss/300):.6f}')
                    time2 = time.time()
                    print(f'Spend time for {300*data.shape[0]} images: {(time2-time1):.6f} sec')
                    time1 = time.time()
                    running_loss = 0.0

            train_loss = np.mean(train_losses)        
            
            
            if valid_data_loader != None:
                self.model.eval()
                valid_metrics = []
                valid_losses = []
                for batch_idx, (data, labels_small, labels) in enumerate(valid_data_loader):
                    data, labels, labels_small = Variable(data), Variable(labels), Variable(labels_small)
                    outputs = self.model(data)
                    # loss вычисляется для сжатых масок для правильной валидации (обучались на сжатых)
                    # чтобы вовремя определить переобучение
                    loss = criterion(outputs, labels_small)
                    valid_losses.append(loss.item())
                    #Преобразуем выход модели к размеру соответствующей маски
                    outputs = F.interpolate(input=outputs, size=(labels.shape[2], labels.shape[3]), mode='bilinear')

                    # метрика считается для исходных размеров потому что именно так итоговое качество
                    # определяется алгоритмом kaggle 
                    metric_value = metric(outputs, labels)
                    valid_metrics.append(metric_value.item())
                    
                valid_loss    = np.mean(valid_losses)
                valid_metric  = np.mean(valid_metrics)
                print(f'Epoch {epoch+1}, train loss: {(train_loss):.6f}, valid_loss: {(valid_loss):.6f}, valid_metric: {(valid_metric):.6f}')
            else:
                print(f'Epoch {epoch+1}, train loss: {(train_loss):.6f}')
                valid_loss = None
                valid_metric = None
            
            epoch_train_losses.append(train_loss)
            epoch_valid_losses.append(valid_loss)
            epoch_valid_metrics.append(valid_metric)
        
        result['epoch_train_losses'] = epoch_train_losses
        result['epoch_valid_losses'] = epoch_valid_losses
        result['epoch_valid_metrics'] = epoch_valid_metrics
        
        return result
    
    
    
    def predict(self, test_data_loader: DataLoader, predict_directory: str=None, output_size: tuple=(1280, 1918), 
                mask_treashold: float=0.5, generate_rle_dataframe: bool=True) -> pd.DataFrame:
        '''Метод для предсказания масок для набора изображения
        Входные параметры:
        test_data_loader: DataLoader - загрузчик данных для предсказания
        predict_directory: str - директория, в которую будут сохраняться сгенерированные маски (если None,
        то маски сохраняться не будут)
        output_size: tuple - пространственная размерность выходных масок
        mask_treashold: float - порог, по которому будет определяться класс каждой точки для масок
        generate_rle_dataframe: bool - флаг, нужна ли генерация rle представлений масок
        Возвращаемые значения:
        rle_dataframe: pd.DataFrame - датафрейм с rle представлениями для масок (если 
        generate_rle_dataframe==True)
        Маски в формате .gif для изображений с соответствующими именами, находятся в директории predict_directory'''
        self.model.eval()
        img_names = []
        img_rles = []
        time1 = time.time()
        time2 = time.time()
        for batch_idx, (index, img, img_name)  in enumerate(test_data_loader):

            img = Variable(img)        
            pred_mask_logit = self.model(img)
            pred_mask_logit = F.interpolate(input=pred_mask_logit, size=output_size, mode='bilinear')
            pred_mask_logit_prob = torch.sigmoid(pred_mask_logit)
            pred_mask = torch.where(pred_mask_logit_prob > mask_treashold, 1, 0)
            
            # Каждое изображение в тензоре преобразуем в картинку и сохраняем
            for i in range(pred_mask.shape[0]):
                if predict_directory != None:
                    mask = (pred_mask[i].cpu().numpy() * 255.0)[0] # [0] - избавляемся от батч размерности
                    PIL_image = Image.fromarray(mask.astype('uint8'), 'L')
                    PIL_image.save((predict_directory+img_name[i]).split('.')[0]+'.gif')
                
                # Если требуется, получаем значения rle для каждой картинки
                if generate_rle_dataframe == True:
                    img_names.append(img_name[i])
                    img_rles.append(NeuralNetwork.tensor_to_rle(pred_mask[i]))
            
            if (batch_idx+1) % 300 == 0:
                    print('-'*50)
                    print(f'Processed images: {(batch_idx+1)*img.shape[0]}')
                    time3 = time.time()
                    print(f'Total time: {(time3-time1):.2f} sec')
                    print(f'Time to process {300*img.shape[0]} images: {(time3-time2):.2f} sec')
                    time2 = time.time()
                
        if generate_rle_dataframe == True:
            rle_dataframe = pd.DataFrame(list(zip(img_names, img_rles)), columns =['img_name', 'img_rle'])
            return rle_dataframe
    
    def save(self, path_to_save: str='./model.pth'):
        '''Метод сохранения весов модели
        Входные параметры:
        path_to_save: str - директория для сохранения состояния модели'''
        torch.save(self.model.state_dict(), path_to_save)
    
    def trace_save(self, path_to_save: str='./model.pth'):
        '''Метод сохранения модели через torchscript
        Входные параметры:
        path_to_save: str - директория для сохранения модели'''
        example_forward_input = torch.rand(1, 3, 512, 512).to('cpu')
        if next(self.model.parameters()).is_cuda:
            example_forward_input= example_forward_input.to('cuda:0')
            
        traced_model = torch.jit.trace((self.model).eval(), example_forward_input)
        torch.jit.save(traced_model, path_to_save)
    
    def onnx_save(self, path_to_save: str='./carvana_model.onnx'):
        '''Метод сохранения модели в формате ONNX
        Входные параметры:
        path_to_save: str - директория для сохранения модели'''
        example_forward_input = torch.randn(1, 3, 1024, 1024, requires_grad=True).to('cpu')
        if next(self.model.parameters()).is_cuda:
            example_forward_input= example_forward_input.to('cuda:0')

        torch.onnx.export(self.model,
                          example_forward_input,
                          path_to_save,
                          export_params=True,
                          opset_version=10,
                          do_constant_folding=True,
                          input_names = ['input'],
                          output_names = ['output'],
                          dynamic_axes={'input' : {0 : 'batch_size'},    # Модель будет работать с произвольным
                                        'output' : {0 : 'batch_size'}})  # размером батча
    
    def load(self, path_to_model: str='./model.pth'):
        '''Метод загрузки весов модели
        Входные параметры:
        path_to_model: str - директория с сохраненными весами модели'''
        self.model.load_state_dict(torch.load(path_to_model))

## Обучение модели

In [14]:
dataset_path = '/home/dima/carvana_dataset'
imgs_path  = dataset_path + '/train/train'
masks_path = dataset_path + '/train_masks/train_masks'

learning_rate = 0.001
num_epochs = 30
mask_treashold = 0.5

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn
preprocess_input = get_preprocessing_fn('mobilenet_v2', pretrained='imagenet')
preprocess_input

functools.partial(<function preprocess_input at 0x7f17a12ca790>, input_space='RGB', input_range=[0, 1], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [16]:
train_transform = A.Compose([
    A.Resize(1024, 1024, cv2.INTER_LINEAR), # для масок автоматически будет применяться своя интерполяция, 
                                          # поэтому на выходе значения маски останутся 0 и 1
    #С нормализацией хуже почему то
    #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=1.0), # согласно imagenet
    #A.Normalize(mean=(0.696, 0.689, 0.684), std=(0.239, 0.243, 0.240), max_pixel_value=1.0), # согласно carvana
    ToTensorV2(),
])

valid_transform = A.Compose([
    A.Resize(1024, 1024, cv2.INTER_LINEAR),
    #С нормализацией хуже почему то
    #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=1.0), # согласно imagenet
    #A.Normalize(mean=(0.696, 0.689, 0.684), std=(0.239, 0.243, 0.240), max_pixel_value=1.0), # согласно carvana
    ToTensorV2(),
])

In [17]:
data = get_data_csv(imgs_path=imgs_path, masks_path=masks_path)
    
# Добавляем признак, по которому будем разбивать датасет на train и test,
# чтобы не было разных фотографий одной и той же машины в двух датасетах
data["car"] = data["file_name"].apply(lambda x: x.split('_')[0])

In [20]:
# Обучение с валидацией
train_df, valid_df = get_train_test(data, separate_feature='car', test_size=0.25)
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)

train_data = CustomDatasetForTrain(train_df, device, train_transform, skip_mask=False)
valid_data = CustomDatasetForTrain(valid_df, device, valid_transform, skip_mask=True)

train_data_loader = DataLoader(train_data, batch_size=2, shuffle=True)
valid_data_loader = DataLoader(valid_data, batch_size=2, shuffle=False)

In [19]:
# Обучение без валидации
train_data = CustomDatasetForTrain(data, device, train_transform, skip_mask=False)
train_data_loader = DataLoader(train_data, batch_size=2, shuffle=True)

In [158]:
# Создаем модель на основе предложенной архитектуры
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16]).to(device)

#model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
#                 encoder_weights='imagenet').to(device)

my_model = NeuralNetwork(model=model)

In [ ]:
#efficientnet-b5

In [19]:
model = smp.DeepLabV3Plus(encoder_name='mobilenet_v2', encoder_depth=5, encoder_weights='imagenet', 
                          encoder_output_stride=16, decoder_channels=256, decoder_atrous_rates=(12, 24, 36), 
                          in_channels=3, classes=1, activation=None, upsampling=4, aux_params=None).to(device)

my_model = NeuralNetwork(model=model)
#summary(model, input_size=(1, 3, 1024, 1024))

In [21]:
criterion = SoftDiceLoss()
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate)
metric = DiceMetric(treashold=mask_treashold)

In [22]:
result = my_model.fit(criterion,
             metric,
             optimizer,
             train_data_loader,
             valid_data_loader,
             epochs=num_epochs)

KeyboardInterrupt: 

In [22]:
# Сохраняем веса обученной модели
my_model.save(path_to_save = './model_lab.pth')

In [23]:
# Сохраняем оттрассированную модель
my_model.trace_save(path_to_save = './model_lab.pt')

In [33]:
# Экспорт модели в onnx
my_model.onnx_save(path_to_save = './carvana_model.onnx')

/home/dima/anaconda3/lib/python3.8/site-packages/torch/onnx/utils.py:88: UserWarning: `enable_onnx_checker' is deprecated and ignored. It will be removed inthe next PyTorch release. To proceed despite ONNX checker failures, youcan catch torch.onnx.ONNXCheckerError.
  warnings.warn("`enable_onnx_checker' is deprecated and ignored. It will be removed in"
/home/dima/anaconda3/lib/python3.8/site-packages/torch/onnx/symbolic_helper.py:381: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator.
  warnings.warn("You are trying to export the model with "

## Загрузка сохраненной модели

In [17]:
# Воспроизводим модель по известной архитектуре и сохраненным весам
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16]).to(device)

my_model = NeuralNetwork(model=model)
my_model.load(path_to_model = './model.pth')

In [15]:
# Загружаем оттрассированную модель
my_model = torch.jit.load('./model.pt')
my_model = NeuralNetwork(model=my_model)
my_model = my_model.to(device)

## Предсказание модели

In [16]:
predict_directory = '/home/dima/carvana_dataset/test/predict_small/'
test_dataset = '/home/dima/carvana_dataset/test/test/'

In [17]:
test_dataframe = {}
test_dataframe['img_addr'] = list(glob.glob(test_dataset + "/*"))
test_dataframe = pd.DataFrame(test_dataframe)
test_dataframe['img_name'] = test_dataframe['img_addr'].apply(lambda x: x.split('/')[-1])

In [21]:
test_data = CustomDatasetForTest(test_dataframe, device, valid_transform)
test_data_loader = DataLoader(test_data, batch_size=2, shuffle=False)

In [29]:
# С сохранением сгенерированных масок в predict_directory
rle_dataframe = my_model.predict(test_data_loader, predict_directory, 
                                 mask_treashold=mask_treashold, generate_rle_dataframe=True)

KeyboardInterrupt: 

In [22]:
# Без сохранения сгенерированных масок в predict_directory
rle_dataframe = my_model.predict(test_data_loader, 
                                 mask_treashold=mask_treashold, generate_rle_dataframe=True)

--------------------------------------------------
Processed images: 600
Total time: 45.61 sec
Time to process 600 images: 45.61 sec
--------------------------------------------------
Processed images: 1200
Total time: 90.61 sec
Time to process 600 images: 44.99 sec
--------------------------------------------------
Processed images: 1800
Total time: 135.49 sec
Time to process 600 images: 44.89 sec
--------------------------------------------------
Processed images: 2400
Total time: 180.41 sec
Time to process 600 images: 44.92 sec
--------------------------------------------------
Processed images: 3000
Total time: 225.76 sec
Time to process 600 images: 45.35 sec
--------------------------------------------------
Processed images: 3600
Total time: 270.78 sec
Time to process 600 images: 45.02 sec
--------------------------------------------------
Processed images: 4200
Total time: 316.20 sec
Time to process 600 images: 45.42 sec
--------------------------------------------------
Process

--------------------------------------------------
Processed images: 37200
Total time: 2785.90 sec
Time to process 600 images: 44.85 sec
--------------------------------------------------
Processed images: 37800
Total time: 2830.82 sec
Time to process 600 images: 44.92 sec
--------------------------------------------------
Processed images: 38400
Total time: 2875.73 sec
Time to process 600 images: 44.91 sec
--------------------------------------------------
Processed images: 39000
Total time: 2920.89 sec
Time to process 600 images: 45.17 sec
--------------------------------------------------
Processed images: 39600
Total time: 2965.87 sec
Time to process 600 images: 44.98 sec
--------------------------------------------------
Processed images: 40200
Total time: 3010.87 sec
Time to process 600 images: 45.00 sec
--------------------------------------------------
Processed images: 40800
Total time: 3055.90 sec
Time to process 600 images: 45.03 sec
-----------------------------------------

--------------------------------------------------
Processed images: 73200
Total time: 5482.15 sec
Time to process 600 images: 45.10 sec
--------------------------------------------------
Processed images: 73800
Total time: 5527.02 sec
Time to process 600 images: 44.87 sec
--------------------------------------------------
Processed images: 74400
Total time: 5571.85 sec
Time to process 600 images: 44.84 sec
--------------------------------------------------
Processed images: 75000
Total time: 5616.82 sec
Time to process 600 images: 44.96 sec
--------------------------------------------------
Processed images: 75600
Total time: 5661.76 sec
Time to process 600 images: 44.95 sec
--------------------------------------------------
Processed images: 76200
Total time: 5706.65 sec
Time to process 600 images: 44.89 sec
--------------------------------------------------
Processed images: 76800
Total time: 5751.62 sec
Time to process 600 images: 44.97 sec
-----------------------------------------

In [23]:
# Получаем датафрейм с результатом для заливки на kaggle
rle_dataframe.to_csv('rle_dataframe.csv', index=True)
sample_submission = pd.read_csv('/home/dima/carvana_dataset/sample_submission.csv')
sample_submission = sample_submission.merge(rle_dataframe, how='left', left_on='img', right_on='img_name')
sample_submission.drop(columns=['rle_mask', 'img_name'], inplace=True)
sample_submission.rename(columns={'img_rle': 'rle_mask'}, inplace=True)
sample_submission.to_csv('submission_07_10.csv', index=False)

In [24]:
torch.cuda.empty_cache()

In [147]:
rle_dataframe.shape

(100064, 2)

## Сравнение интерполяций

### Попробовать bicubic

In [25]:
iterator = iter(valid_data_loader)

In [26]:
input_tensor = iterator.next()

In [27]:
input_tensor[0].shape

torch.Size([2, 3, 1024, 1024])

In [28]:
input_tensor[1].shape

torch.Size([2, 1, 1024, 1024])

In [29]:
input_tensor[2].shape

torch.Size([2, 1, 1280, 1918])

In [30]:
out = my_model.model(input_tensor[0])

In [31]:
out.shape

torch.Size([2, 1, 1024, 1024])

In [33]:
output_nearest = F.interpolate(input=out, size=(1280, 1918), mode='nearest')
output_bilinear = F.interpolate(input=out, size=(1280, 1918), mode='bilinear', align_corners=True)
#output_linear = F.interpolate(input=out, size=(1280, 1918), mode='linear')  

In [34]:
dice = DiceMetric()
dice_nearest = dice(output_nearest, input_tensor[2])
dice_nearest

tensor(0.9945, device='cuda:0')

In [35]:
dice_bilinear = dice(output_bilinear, input_tensor[2])
dice_bilinear

tensor(0.9955, device='cuda:0')

In [57]:
output_nearest = torch.sigmoid(output_nearest)
output_bilinear = torch.sigmoid(output_bilinear)

In [58]:
output_nearest = torch.where(output_nearest > 0.5, 1, 0)
output_bilinear = torch.where(output_bilinear > 0.5, 1, 0)

In [59]:
output_nearest.shape

torch.Size([2, 1, 1280, 1918])

In [60]:
predict_directory = '/home/dima/carvana_dataset/test/predict_small/'

In [61]:
output_nearest = (output_nearest[0].cpu().numpy() * 255.0)[0] # [0] - избавляемся от батч размерности
output_nearest = Image.fromarray(output_nearest.astype('uint8'), 'L')
output_nearest.save((predict_directory+'111').split('.')[0]+'.gif')

In [62]:
output_bilinear = (output_bilinear[0].cpu().numpy() * 255.0)[0] # [0] - избавляемся от батч размерности
output_bilinear = Image.fromarray(output_bilinear.astype('uint8'), 'L')
output_bilinear.save((predict_directory+'222').split('.')[0]+'.gif')